# Real-Time Personalized News Recommendation
A complete Colab notebook with an interactive UI sidebar for recommended news articles.

In [1]:
# Install required packages
!pip install newsapi-python sentence-transformers pandas ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [2]:
import os
import pandas as pd
import math
import numpy as np
from datetime import datetime, timezone
from newsapi import NewsApiClient
from sentence_transformers import SentenceTransformer

## Configuration
Set your NewsAPI key as an environment variable or directly below.

In [3]:
import os
from newsapi import NewsApiClient
from sentence_transformers import SentenceTransformer

# Option 1: set via environment variable in Colab
# os.environ['NEWSAPI_KEY'] = 'YOUR_KEY_HERE'

# Option 2: directly assign (less secure)
NEWSAPI_KEY = os.getenv('NEWSAPI_KEY', '5dc3d16fe871435685839cc0e4a6da3f')
newsapi = NewsApiClient(api_key=NEWSAPI_KEY)
embedder = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# Configuration
# Option 1: set via environment variable in Colab
# import os
# os.environ['NEWSAPI_KEY'] = 'YOUR_KEY_HERE'

# Option 2: directly assign (less secure)
NEWSAPI_KEY = os.getenv('NEWSAPI_KEY', '5dc3d16fe871435685839cc0e4a6da3f')
newsapi = NewsApiClient(api_key=NEWSAPI_KEY)
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
import os
import pandas as pd
import math
import numpy as np
from datetime import datetime, timezone
from newsapi import NewsApiClient
from sentence_transformers import SentenceTransformer

# Configuration
NEWSAPI_KEY = os.getenv('NEWSAPI_KEY', '5dc3d16fe871435685839cc0e4a6da3f')
newsapi = NewsApiClient(api_key=NEWSAPI_KEY)
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Fetch articles
articles = []

# Top-headlines for sports & entertainment
for cat in ['sports', 'entertainment']:
    resp = newsapi.get_top_headlines(category=cat, language='en', page_size=50)
    for art in resp.get('articles', []):
        text = art.get('content') or art.get('description') or ''
        articles.append({
            'title': art.get('title'),
            'url': art.get('url'),
            'category': cat,
            'published_at': art.get('publishedAt'),
            'text': text
        })

# Politics via everything endpoint
resp = newsapi.get_everything(
    q='politics',
    language='en',
    page_size=50,
    sort_by='publishedAt'
)
for art in resp.get('articles', []):
    text = art.get('content') or art.get('description') or ''
    articles.append({
        'title': art.get('title'),
        'url': art.get('url'),
        'category': 'politics',
        'published_at': art.get('publishedAt'),
        'text': text
    })

# Create DataFrame
df = pd.DataFrame(articles)
df['published_at'] = pd.to_datetime(df['published_at'])
df['age_hours'] = (
    datetime.now(timezone.utc)
    - df['published_at']
).dt.total_seconds() / 3600

# Preview
df.head()


,title,url,category,published_at,text,age_hours
0,"Antonio Gates makes history, becoming Hall of ...",https://www.nbcsports.com/nfl/profootballtalk/...,sports,2025-08-02 19:26:44+00:00,"Antonio Gates made history Saturday, becoming ...",26.413203
1,Ohio State Practice Observations: Julian Sayin...,https://www.elevenwarriors.com/ohio-state-foot...,sports,2025-08-02 19:15:49+00:00,Ohio State saved its best of its three open pr...,26.595148
2,Brian Schottenheimer doesn’t believe the Micah...,https://www.nbcsports.com/nfl/profootballtalk/...,sports,2025-08-02 19:13:52+00:00,"In making a trade request, Cowboys linebacker ...",26.627648
3,Volatility at center of Yankees’ overhaul lead...,https://nypost.com/2025/08/02/sports/volatilit...,sports,2025-08-02 19:03:00+00:00,A few thoughts coming out of the trade deadlin...,26.808759
4,"Sterling Sharpe honors his brother, Shannon, i...",https://www.nbcsports.com/nfl/profootballtalk/...,sports,2025-08-02 18:32:25+00:00,If Sterling Sharpe had played more than seven ...,27.318481


## Fetch Articles
- Sports & Entertainment via top-headlines
- Politics via the everything endpoint

## Compute Embeddings

In [6]:
# Compute text embeddings
df['embedding'] = embedder.encode(
    df['text'].tolist(),
    normalize_embeddings=True
).tolist()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


## Build User Profile Embedding

In [7]:
# Define user category weights
user_weights = {'sports': 0.5, 'politics': 0.3, 'entertainment': 0.2}

# Compute centroids
centroids = {}
for cat, w in user_weights.items():
    embs = [emb for emb, c in zip(df['embedding'], df['category']) if c == cat]
    if embs:
        centroids[cat] = np.mean(embs, axis=0)

# Weighted average profile
user_profile = sum(w * centroids[cat] for cat, w in user_weights.items() if cat in centroids)

## Score and Select Top Articles

In [8]:
# Score articles
scores = []
for _, row in df.iterrows():
    cat_score = user_weights.get(row['category'], 0)
    freshness = math.exp(-0.1 * row['age_hours'])
    sim = np.dot(user_profile, row['embedding'])
    score = 0.5 * cat_score + 0.3 * freshness + 0.2 * sim
    scores.append(score)

df['score'] = scores
topk = df.sort_values('score', ascending=False).head(20).reset_index(drop=True)
topk[['title','category','published_at','score']]

,title,category,published_at,score
0,Brian Schottenheimer doesn’t believe the Micah...,sports,2025-08-02 19:13:52+00:00,0.303150
1,Marcus Stroman gives one-word message in respo...,sports,2025-08-02 17:16:00+00:00,0.300214
2,Volatility at center of Yankees’ overhaul lead...,sports,2025-08-02 19:03:00+00:00,0.299093
3,Hall of Fame Enshrinement: 2025 class takes it...,sports,2025-08-02 17:53:00+00:00,0.294287
4,Ohio State Practice Observations: Julian Sayin...,sports,2025-08-02 19:15:49+00:00,0.292395
5,England vs India: Zak Crawley falls late with ...,sports,2025-08-02 18:20:18+00:00,0.292032
6,‘I’m absolutely useless’ – Hamilton says Ferra...,sports,2025-08-02 16:20:30+00:00,0.291869
7,‘It’s super disrespectful’: WNBA players exasp...,sports,2025-08-02 15:59:00+00:00,0.291864
8,Connecticut Sun will reportedly be sold for $3...,sports,2025-08-02 17:42:47+00:00,0.291356
9,Red Sox starter Tanner Houck to undergo Tommy ...,sports,2025-08-02 17:17:00+00:00,0.289208


## Interactive UI Sidebar

In [9]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Create buttons for each recommended article
buttons = []
for idx, row in topk.iterrows():
    btn = widgets.Button(description=row['title'], layout=widgets.Layout(width='auto'))
    buttons.append(btn)

# Sidebar and content panes
sidebar = widgets.VBox(buttons, layout=widgets.Layout(width='30%', height='600px', overflow='auto'))
content = widgets.Output(layout=widgets.Layout(width='70%', padding='10px'))

# Button click handler
def on_button_click(btn):
    idx = buttons.index(btn)
    art = topk.iloc[idx]
    content.clear_output()
    with content:
        display(HTML(f"<h2>{art['title']}</h2>"))
        display(HTML(f"<p><em>{art['category']} | {art['published_at']}</em></p>"))
        display(HTML(f"<p>{art['text']}</p>"))
        display(HTML(f"<p><a href='{art['url']}' target='_blank'>Read full article</a></p>"))

for btn in buttons:
    btn.on_click(on_button_click)

# Display the UI
display(widgets.HBox([sidebar, content]))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Share via Binder + Voilà
1. Push this notebook to GitHub in a public repo.
2. Include `requirements.txt` with:
```
newsapi-python
sentence-transformers
pandas
ipywidgets
```
3. Use the Binder URL:
```
https://mybinder.org/v2/gh/<YourUser>/<YourRepo>/main?urlpath=voila/render/<NotebookName>.ipynb
```

In [10]:
import os
print(os.getcwd())

/content


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# List what’s under /content/drive
!ls /content/drive

MyDrive  Othercomputers


In [14]:
# List what’s under /content/drive/MyDrive
!ls /content/drive/MyDrive

'Activity Exemplar: Course 3 Automatidata Executive Summary.gslides'
'Activity Exemplar: Course 6 Automatidata Executive Summary.gslides'
'Activity Exemplar: Customer personas.gdoc'
'Activity Exemplar: Hootsuite Tweets and Planner.gdoc'
'Activity Exemplar: Job search project plan.gsheet'
'Activity Exemplar: Product Backlog.gsheet'
'Activity Exemplar: Respond to customer comments.gdoc'
'Activity Exemplar: Responsive search ad organizer.gsheet'
'Activity Exemplar: Social listening insights.gdoc'
'Activity Exemplar: Social media calendar .gsheet'
'Activity Exemplar: TikTok Course 4 executive summary.gslides'
'Activity Exemplar: Waze Course 4 executive summary.gslides'
'Activity Exemplar: Webpage titles and meta descriptions.gdoc'
'Activity Template: Course 4 PACE strategy document.gdoc'
'Activity Template: Course 6 PACE strategy document.gdoc'
'Activity Template: Customer personas.gdoc'
'Activity Template: Job search project plan.gsheet'
'Activity Template: RACI Matrix.gdoc'
'Activity Tem

In [15]:
# Finally list your Colab Notebooks folder
!ls "/content/drive/MyDrive/Colab Notebooks"

 00-Large-Language-Models.ipynb
 01_importing_tensorflow.ipynb
'02_02 (1).ipynb'
'02_02 (2).ipynb'
'02_02 (3).ipynb'
 02_02.ipynb
'02_02selfcheck (1).ipynb'
'02_02selfcheck (2).ipynb'
'02_02selfcheck (3).ipynb'
 02_02selfcheck.ipynb
'02_03 (1).ipynb'
'02_03 (2).ipynb'
 02_03.ipynb
 02_03selfcheck.ipynb
'02_04 (1).ipynb'
 02_04.ipynb
 02_04selfcheck.ipynb
'02_05 (1).ipynb'
 02_05.ipynb
 02_05selfcheck.ipynb
'02_06 (1).ipynb'
 02_06.ipynb
 02_06selfcheck.ipynb
 02_07.ipynb
 02_08.ipynb
'02_09 (1).ipynb'
 02_09.ipynb
 02_09selfcheck.ipynb
 02_defining_tensors.ipynb
 03_03selfcheck.ipynb
'03_05 (1).ipynb'
 03_05.ipynb
 03_05selfcheck.ipynb
 03_06.ipynb
 03_06selfcheck.ipynb
'03_07 (1).ipynb'
 03_07.ipynb
'03_07selfcheck (1).ipynb'
 03_07selfcheck.ipynb
'03_08 (1).ipynb'
 03_08.ipynb
'03_08selfcheck (1).ipynb'
 03_08selfcheck.ipynb
'03_09 (1).ipynb'
 03_09.ipynb
'03_09selfcheck (1).ipynb'
'03_09selfcheck (2).ipynb'
 03_09selfcheck.ipynb
'03_10selfcheck (1).ipynb'
 03_10selfcheck.ipynb
'03_1

In [16]:
%cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [18]:
%cd /content
!git clone https://github.com/aiserhucui/news-recommendation-demo.git repo


/content
Cloning into 'repo'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 2), reused 6 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 27.81 KiB | 1.54 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [19]:
# 1. Enter your repo directory
%cd /content/repo

/content/repo


In [20]:
# 2. Configure Git (only needed once per Colab session)
!git config user.email "aiserhucui@gmail.com"
!git config user.name  "Aiser Hucui"

In [21]:
# 3. Copy your updated notebook from Drive into the repo
!cp "/content/drive/MyDrive/Colab Notebooks/news_recommendation_complete_colab.ipynb" .
